# FPUS23 YOLO First Run (Colab)

This notebook prepares the FPUS23 dataset and runs a safe first training (no custom anchors, no balancing).
For advanced runs (anchors, balancing, denoiser), use the full guide later.

In [ ]:
# Clone repo and move into it
!git clone https://github.com/Srinivas-Raghav-VC/MultiFetalOrgan-Detection.git /content/fpus23
%cd /content/fpus23
!python --version
import torch; print('GPU:', torch.cuda.is_available())

In [ ]:
# Optional: save results to Google Drive
SAVE_TO_DRIVE = True
DRIVE_RESULTS = '/content/drive/MyDrive/FPUS23_runs'
if SAVE_TO_DRIVE:
    from google.colab import drive; drive.mount('/content/drive', force_remount=True)
    import pathlib as p
    p.Path(DRIVE_RESULTS).mkdir(parents=True, exist_ok=True)
    print('Results will be saved to', DRIVE_RESULTS)
else:
    print('Saving to local /content/fpus23_project/runs')


In [ ]:
# Install pinned dependencies
!pip -q install -r /content/fpus23/requirements_colab.txt
!pip -q install gdown
# Recommend batch size based on GPU
import torch, os
gpu = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'
print('GPU:', gpu)
# Simple heuristic for T4/A100
if 'T4' in gpu: os.environ['FPUS23_BATCH']='16'
elif 'A100' in gpu: os.environ['FPUS23_BATCH']='24'
else: os.environ['FPUS23_BATCH']='12'
print('Recommended batch:', os.environ['FPUS23_BATCH'])


In [ ]:
# Download and unzip dataset (public Drive link)
!gdown https://drive.google.com/uc?id=1LL-r2hNiP6C190UBSE4v1FFCF3OQT9N3 -O /content/FPUS23_Dataset.zip
!unzip -q /content/FPUS23_Dataset.zip -d /content/FPUS23_Dataset
!ls -lh /content/FPUS23_Dataset

If the public download fails, mount Drive and copy your ZIP:

In [ ]:
# Alternative: mount Drive and copy
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/FPUS23_Dataset.zip' /content/ || true
!unzip -q /content/FPUS23_Dataset.zip -d /content/FPUS23_Dataset || true

In [ ]:
# Prepare dataset (XML -> YOLO -> COCO)
!python '/content/fpus23/scripts/prepare_fpus23.py' \
  --dataset-root /content/FPUS23_Dataset/Dataset \
  --project-root /content/fpus23_project
!ls -lh /content/fpus23_project/dataset/fpus23_yolo/

In [ ]:
# Sanity-check YOLO dataset (val split)
!python '/content/fpus23/scripts/tools/verify_yolo_dataset.py' \
  --data /content/fpus23_project/dataset/fpus23_yolo/data.yaml \
  --split val \
  --vis-out /content/tmp/vis_val \
  --limit 24
!ls -lh /content/tmp/vis_val | head

In [ ]:
# First training (no anchors, no balancing)
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' \
  --data /content/fpus23_project/dataset/fpus23_yolo/data.yaml \
  --model yolo11s.pt \
  --epochs 100 \
  --batch $FPUS23_BATCH \
  --imgsz 768 \
  --project $DRIVE_RESULTS \
  --name fpus23_colab_phase1
!ls -lh /content/fpus23_project/runs/detect/fpus23_colab_phase1/weights/


In [ ]:
# Evaluate model
from ultralytics import YOLO
model = YOLO('/content/fpus23_project/runs/detect/fpus23_colab_phase1/weights/best.pt')
results = model.val(data='/content/fpus23_project/dataset/fpus23_yolo/data.yaml')
results

## Optional: train a medium model (yolo11m) if GPU allows

In [ ]:
# yolo11m often gives +0.5–1.5 mAP over 11s; adjust batch if OOM
!python 'New folder/scripts/train_yolo_fpus23_phase1.py' \
  --data /content/fpus23_project/dataset/fpus23_yolo/data.yaml \
  --model yolo11m.pt \
  --epochs 100 \
  --batch 8 \
  --imgsz 768 \
  --project $DRIVE_RESULTS \
  --name fpus23_colab_phase1_m
!ls -lh /content/fpus23_project/runs/detect/fpus23_colab_phase1_m/weights/ || true
